In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path

import pymedphys
import pymedphys._wlutz.findfield
import pymedphys._wlutz.iview
import pymedphys._wlutz.imginterp
import pymedphys._wlutz.reporting
import pymedphys._wlutz.interppoints

In [ ]:
bb_diameter = 8
edge_lengths = [20, 24]
penumbra = 2

In [ ]:
training_data_paths = pymedphys.zenodo_data_paths('wlutz_tensorflow_training_data')

In [ ]:
image_paths = {path.stem: path for path in training_data_paths if path.suffix == '.png'}
labels_path = [path for path in training_data_paths if path.suffix == '.json'][0]

In [ ]:
with open(labels_path, 'r') as labels_file:
    labels = json.load(labels_file)

In [ ]:
keys = list(image_paths.keys())

In [ ]:
key = keys[2]
image_path = image_paths[key]
x, y, img = pymedphys._wlutz.iview.iview_image_transform(image_path)

In [ ]:
label = labels[key]['pymedphys']
label

In [ ]:
pymedphys._wlutz.reporting.image_analysis_figure(
    x, y, img,
    label['bb_centre'], label['field_centre'], label['field_rotation'],
    bb_diameter, edge_lengths, penumbra
);

In [ ]:
centre_of_mass = pymedphys._wlutz.findfield.get_centre_of_mass(x, y, img)
field = pymedphys._wlutz.imginterp.create_interpolated_field(x, y, img)

In [ ]:
dx = 1/8
np.sqrt(20**2 + 24**2) / 2

In [ ]:
centre_of_mass

In [ ]:
x_about_zero = np.arange(-20, 20+dx, dx)
y_about_zero = np.arange(-20, 20+dx, dx)

In [ ]:
x_interp = x_about_zero + centre_of_mass[0]
y_interp = y_about_zero + centre_of_mass[1]

xx, yy = np.meshgrid(x_interp, y_interp)

In [ ]:
interpolated_image = field(xx, yy)

In [ ]:
# interpolated_image

In [ ]:
plt.pcolormesh(xx, yy, interpolated_image)
plt.axis('equal')

In [ ]:
field_centre = np.array(label['field_centre']) - np.array(centre_of_mass)
field_centre

In [ ]:
field_rotation = label['field_rotation']
field_rotation

In [ ]:
bb_centre = np.array(label['bb_centre']) - np.array(centre_of_mass)
bb_centre

In [ ]:
xx_about_zero, yy_about_zero = np.meshgrid(x_about_zero, y_about_zero)
segmentation_mask = np.zeros_like(interpolated_image)

In [ ]:
field_transform = pymedphys._wlutz.interppoints.translate_and_rotate_transform(field_centre, field_rotation)
rect_dx = [-edge_lengths[0] / 2, 0, edge_lengths[0], 0, -edge_lengths[0]]
rect_dy = [-edge_lengths[1] / 2, edge_lengths[1], 0, -edge_lengths[1], 0]

draw_x = np.cumsum(rect_dx)
draw_y = np.cumsum(rect_dy)

rect_x, rect_y = pymedphys._wlutz.interppoints.apply_transform(draw_x, draw_y, field_transform)
rect_points = list(zip(rect_x, rect_y))

In [ ]:
rectangle = matplotlib.path.Path(rect_points)

points = np.swapaxes(np.vstack([xx_about_zero.ravel(), yy_about_zero.ravel()]), 0, 1)
rectangle_mask = rectangle.contains_points(points).reshape(len(yy_about_zero), len(xx_about_zero))

segmentation_mask[rectangle_mask] = 1

within_bb = np.sqrt((xx_about_zero - bb_centre[0])**2 + (yy_about_zero - bb_centre[1])**2) <= bb_diameter/2

segmentation_mask[within_bb] = 2

In [ ]:
pymedphys._wlutz.reporting.image_analysis_figure(
    x_about_zero, y_about_zero, interpolated_image,
    bb_centre, field_centre, field_rotation,
    bb_diameter, edge_lengths, penumbra
);

plt.contourf(xx_about_zero, yy_about_zero, segmentation_mask, alpha=0.5, cmap='bwr')